In [1]:
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sqlalchemy.engine import URL
from sqlalchemy import create_engine

In [2]:
load_dotenv(dotenv_path='./data/.env')

True

In [3]:
# Configurar parametros de coneccion a la base de datos
url_object = URL.create(
    'postgresql+psycopg2',
    username=os.getenv('USER_NAME'),
    password=os.getenv('PASSWORD'),
    host=os.getenv('HOST'),
    database=os.getenv('DB_NAME')
)

In [4]:
# Crear motor con SQLAlchemy para conectarse a la base de datos
engine = create_engine(url_object)

'supermercado_dw'

# Extraer datos

In [5]:
data = pd.read_csv(
    'customer_shopping_data.csv', 
    sep=',',
    dtype={
        'invoice_no': str,
        'customer_id': str,
        'gender': str,
        'age': int,
        'category': str,
        'quantity': int,
        'price': float,
        'payment_method': str,
        'shopping_mall': str
    }
)

In [6]:
data

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I138884,C241288,Female,28,Clothing,5,1500.40,Credit Card,5/8/2022,Kanyon
1,I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,12/12/2021,Forum Istanbul
2,I127801,C266599,Male,20,Clothing,1,300.08,Cash,9/11/2021,Metrocity
3,I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,16/05/2021,Metropol AVM
4,I337046,C189076,Female,53,Books,4,60.60,Cash,24/10/2021,Kanyon
...,...,...,...,...,...,...,...,...,...,...
99452,I219422,C441542,Female,45,Souvenir,5,58.65,Credit Card,21/09/2022,Kanyon
99453,I325143,C569580,Male,27,Food & Beverage,2,10.46,Cash,22/09/2021,Forum Istanbul
99454,I824010,C103292,Male,63,Food & Beverage,2,10.46,Debit Card,28/03/2021,Metrocity
99455,I702964,C800631,Male,56,Technology,4,4200.00,Cash,16/03/2021,Istinye Park


# Transformar datos

In [7]:
# Ajuste del campo de fecha
data['invoice_date'] = data['invoice_date'].apply(lambda x: pd.Timestamp(x).date())

In [8]:
# Reemplazar espacios en blanco en el dataset inicial por NaN
data = data.replace(r'^\s+$', np.nan, regex=True)

In [9]:
# Ajuste de nombre de las columnas
data = data.rename(columns={
    'invoice_no': 'id_factura',
    'customer_id': 'id_cliente',
    'gender': 'genero',
    'age': 'edad',
    'category': 'categoria',
    'quantity': 'cantidad',
    'price': 'precio_unitario',
    'payment_method': 'metodo_pago',
    'invoice_date': 'fecha_factura',
    'shopping_mall': 'centro_comercial'
})

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99457 entries, 0 to 99456
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id_factura        99457 non-null  object 
 1   id_cliente        99457 non-null  object 
 2   genero            99457 non-null  object 
 3   edad              99457 non-null  int64  
 4   categoria         99457 non-null  object 
 5   cantidad          99457 non-null  int64  
 6   precio_unitario   99457 non-null  float64
 7   metodo_pago       99457 non-null  object 
 8   fecha_factura     99457 non-null  object 
 9   centro_comercial  99457 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 7.6+ MB


In [11]:
num_cols = ['edad', 'cantidad', 'precio_unitario']

cat_cols = [
    'genero',
    'categoria',
    'metodo_pago',
    'centro_comercial'
]

valores_validos = {
    'genero': ['Female', 'Male'],
    'categoria': ['Clothing', 'Shoes', 'Books', 'Cosmetics', 'Food & Beverage', 'Toys', 'Technology', 'Souvenir'],
    'metodo_pago': ['Credit Card', 'Debit Card', 'Cash'],
    'centro_comercial': [
        'Kanyon',
        'Forum Istanbul',
        'Metrocity',
        'Metropol AVM',
        'Istinye Park',
        'Mall of Istanbul',
        'Emaar Square Mall',
        'Cevahir AVM', 
        'Viaport Outlet', 
        'Zorlu Center'
    ]
}

In [12]:
# Se convierten en nulos los valores atípicos identificados
for var in cat_cols:
    data[var] = data[var].apply(lambda x: x if x in valores_validos[var] else np.nan)

data['edad'] = np.where((data['edad'] >= 18) & (data['edad'] <= 70), data['edad'], np.nan)

In [13]:
# Imputación por la media y moda

# Imputacion de variables numéricas
imp_numericas = SimpleImputer(missing_values=np.nan, strategy='mean')
data[num_cols] = imp_numericas.fit_transform(data[num_cols])

# Imputacion de variables categóricas
imp_categoricas = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
data[cat_cols] = imp_categoricas.fit_transform(data[cat_cols])

data['edad'] = data['edad'].apply(int)
data['cantidad'] = data['cantidad'].apply(int)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99457 entries, 0 to 99456
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id_factura        99457 non-null  object 
 1   id_cliente        99457 non-null  object 
 2   genero            99457 non-null  object 
 3   edad              99457 non-null  int64  
 4   categoria         99457 non-null  object 
 5   cantidad          99457 non-null  int64  
 6   precio_unitario   99457 non-null  float64
 7   metodo_pago       99457 non-null  object 
 8   fecha_factura     99457 non-null  object 
 9   centro_comercial  99457 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 7.6+ MB


In [14]:
data

,id_factura,id_cliente,genero,edad,categoria,cantidad,precio_unitario,metodo_pago,fecha_factura,centro_comercial
0,I138884,C241288,Female,28,Clothing,5,1500.40,Credit Card,2022-05-08,Kanyon
1,I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,2021-12-12,Forum Istanbul
2,I127801,C266599,Male,20,Clothing,1,300.08,Cash,2021-09-11,Metrocity
3,I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,2021-05-16,Metropol AVM
4,I337046,C189076,Female,53,Books,4,60.60,Cash,2021-10-24,Kanyon
...,...,...,...,...,...,...,...,...,...,...
99452,I219422,C441542,Female,45,Souvenir,5,58.65,Credit Card,2022-09-21,Kanyon
99453,I325143,C569580,Male,27,Food & Beverage,2,10.46,Cash,2021-09-22,Forum Istanbul
99454,I824010,C103292,Male,63,Food & Beverage,2,10.46,Debit Card,2021-03-28,Metrocity
99455,I702964,C800631,Male,56,Technology,4,4200.00,Cash,2021-03-16,Istinye Park


In [15]:
# Mapear datos al idioma español
genero_esp = {'Female': 'Femenino', 'Male': 'Masculino'}

categoria_esp = {
    'Clothing': 'Ropa', 
    'Shoes': 'Zapatos',
    'Books': 'Libros',
    'Cosmetics': 'Cosméticos',
    'Food & Beverage': 'Alimentos y bebidas',
    'Toys': 'Juguetes',
    'Technology': 'Tecnología',
    'Souvenir': 'Recuerdos'
}

metodo_pago_esp = {
    'Credit Card': 'Tarjeta de crédito', 
    'Debit Card': 'Tarjeta de débito',
    'Cash': 'Efectivo'
}

In [16]:
data['genero'] = data['genero'].map(genero_esp)
data['categoria'] = data['categoria'].map(categoria_esp)
data['metodo_pago'] = data['metodo_pago'].map(metodo_pago_esp)

data

,id_factura,id_cliente,genero,edad,categoria,cantidad,precio_unitario,metodo_pago,fecha_factura,centro_comercial
0,I138884,C241288,Femenino,28,Ropa,5,1500.40,Tarjeta de crédito,2022-05-08,Kanyon
1,I317333,C111565,Masculino,21,Zapatos,3,1800.51,Tarjeta de débito,2021-12-12,Forum Istanbul
2,I127801,C266599,Masculino,20,Ropa,1,300.08,Efectivo,2021-09-11,Metrocity
3,I173702,C988172,Femenino,66,Zapatos,5,3000.85,Tarjeta de crédito,2021-05-16,Metropol AVM
4,I337046,C189076,Femenino,53,Libros,4,60.60,Efectivo,2021-10-24,Kanyon
...,...,...,...,...,...,...,...,...,...,...
99452,I219422,C441542,Femenino,45,Recuerdos,5,58.65,Tarjeta de crédito,2022-09-21,Kanyon
99453,I325143,C569580,Masculino,27,Alimentos y bebidas,2,10.46,Efectivo,2021-09-22,Forum Istanbul
99454,I824010,C103292,Masculino,63,Alimentos y bebidas,2,10.46,Tarjeta de débito,2021-03-28,Metrocity
99455,I702964,C800631,Masculino,56,Tecnología,4,4200.00,Efectivo,2021-03-16,Istinye Park


In [18]:
# Calcular total por cada transaccion
data['venta_total'] = data['cantidad'] * data['precio_unitario']

In [19]:
data

,id_factura,id_cliente,genero,edad,categoria,cantidad,precio_unitario,metodo_pago,fecha_factura,centro_comercial,venta_total
0,I138884,C241288,Femenino,28,Ropa,5,1500.40,Tarjeta de crédito,2022-05-08,Kanyon,7502.00
1,I317333,C111565,Masculino,21,Zapatos,3,1800.51,Tarjeta de débito,2021-12-12,Forum Istanbul,5401.53
2,I127801,C266599,Masculino,20,Ropa,1,300.08,Efectivo,2021-09-11,Metrocity,300.08
3,I173702,C988172,Femenino,66,Zapatos,5,3000.85,Tarjeta de crédito,2021-05-16,Metropol AVM,15004.25
4,I337046,C189076,Femenino,53,Libros,4,60.60,Efectivo,2021-10-24,Kanyon,242.40
...,...,...,...,...,...,...,...,...,...,...,...
99452,I219422,C441542,Femenino,45,Recuerdos,5,58.65,Tarjeta de crédito,2022-09-21,Kanyon,293.25
99453,I325143,C569580,Masculino,27,Alimentos y bebidas,2,10.46,Efectivo,2021-09-22,Forum Istanbul,20.92
99454,I824010,C103292,Masculino,63,Alimentos y bebidas,2,10.46,Tarjeta de débito,2021-03-28,Metrocity,20.92
99455,I702964,C800631,Masculino,56,Tecnología,4,4200.00,Efectivo,2021-03-16,Istinye Park,16800.00


In [21]:
# Asignar claves foráneas para cargar datos a la tabla de (ventas) hechos
ids_cetegoria = {
    'Ropa': 1, 
    'Zapatos': 2,
    'Libros': 3,
    'Cosméticos': 4,
    'Alimentos y bebidas': 5,
    'Juguetes': 6,
    'Tecnología': 7,
    'Recuerdos': 8
}

ids_centro_comercial = {
    'Kanyon': 1,
    'Forum Istanbul': 2,
    'Metrocity': 3,
    'Metropol AVM': 4,
    'Istinye Park': 5,
    'Mall of Istanbul': 6,
    'Emaar Square Mall': 7,
    'Cevahir AVM': 8, 
    'Viaport Outlet': 9, 
    'Zorlu Center': 10
}

ids_metodo_pago = {
    'Tarjeta de crédito': 1, 
    'Tarjeta de débito': 2,
    'Efectivo': 3
}

In [22]:
# Leer datos desde la dimensión de tiempo
sql_query = 'SELECT fecha_completa, id FROM fecha'

# Realizar la lectura de los datos
with engine.begin() as conn:
    df_fechas = pd.read_sql(sql=sql_query, con=conn)

In [23]:
df_fechas = df_fechas.rename(columns={'id': 'id_fecha'})
df_fechas['fecha_completa'] = df_fechas['fecha_completa'].apply(lambda x: x.strftime('%Y-%m-%d'))
df_fechas

,fecha_completa,id_fecha
0,2010-01-01,1
1,2010-01-02,2
2,2010-01-03,3
3,2010-01-04,4
4,2010-01-05,5
...,...,...
7665,2030-12-27,7666
7666,2030-12-28,7667
7667,2030-12-29,7668
7668,2030-12-30,7669


In [24]:
df_ventas = data.copy()

df_ventas['id_categoria'] = df_ventas['categoria'].map(ids_cetegoria)
df_ventas['id_metodo_pago'] = df_ventas['metodo_pago'].map(ids_metodo_pago)
df_ventas['id_centro_comercial'] = df_ventas['centro_comercial'].map(ids_centro_comercial)
df_ventas['fecha_factura'] = df_ventas['fecha_factura'].apply(lambda x: x.strftime('%Y-%m-%d'))
df_ventas = pd.merge(
    df_ventas,
    df_fechas,
    how='left',
    left_on='fecha_factura',
    right_on='fecha_completa'
)
df_ventas

,id_factura,id_cliente,genero,edad,categoria,cantidad,precio_unitario,metodo_pago,fecha_factura,centro_comercial,venta_total,id_categoria,id_metodo_pago,id_centro_comercial,fecha_completa,id_fecha
0,I138884,C241288,Femenino,28,Ropa,5,1500.40,Tarjeta de crédito,2022-05-08,Kanyon,7502.00,1,1,1,2022-05-08,4511
1,I317333,C111565,Masculino,21,Zapatos,3,1800.51,Tarjeta de débito,2021-12-12,Forum Istanbul,5401.53,2,2,2,2021-12-12,4364
2,I127801,C266599,Masculino,20,Ropa,1,300.08,Efectivo,2021-09-11,Metrocity,300.08,1,3,3,2021-09-11,4272
3,I173702,C988172,Femenino,66,Zapatos,5,3000.85,Tarjeta de crédito,2021-05-16,Metropol AVM,15004.25,2,1,4,2021-05-16,4154
4,I337046,C189076,Femenino,53,Libros,4,60.60,Efectivo,2021-10-24,Kanyon,242.40,3,3,1,2021-10-24,4315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99452,I219422,C441542,Femenino,45,Recuerdos,5,58.65,Tarjeta de crédito,2022-09-21,Kanyon,293.25,8,1,1,2022-09-21,4647
99453,I325143,C569580,Masculino,27,Alimentos y bebidas,2,10.46,Efectivo,2021-09-22,Forum Istanbul,20.92,5,3,2,2021-09-22,4283
99454,I824010,C103292,Masculino,63,Alimentos y bebidas,2,10.46,Tarjeta de débito,2021-03-28,Metrocity,20.92,5,2,3,2021-03-28,4105
99455,I702964,C800631,Masculino,56,Tecnología,4,4200.00,Efectivo,2021-03-16,Istinye Park,16800.00,7,3,5,2021-03-16,4093


In [25]:
df_ventas = df_ventas[[
    'id_factura', 
    'id_cliente', 
    'id_categoria', 
    'id_metodo_pago', 
    'id_fecha', 
    'id_centro_comercial', 
    'precio_unitario', 
    'cantidad',
    'venta_total'
]]
df_ventas

,id_factura,id_cliente,id_categoria,id_metodo_pago,id_fecha,id_centro_comercial,precio_unitario,cantidad,venta_total
0,I138884,C241288,1,1,4511,1,1500.40,5,7502.00
1,I317333,C111565,2,2,4364,2,1800.51,3,5401.53
2,I127801,C266599,1,3,4272,3,300.08,1,300.08
3,I173702,C988172,2,1,4154,4,3000.85,5,15004.25
4,I337046,C189076,3,3,4315,1,60.60,4,242.40
...,...,...,...,...,...,...,...,...,...
99452,I219422,C441542,8,1,4647,1,58.65,5,293.25
99453,I325143,C569580,5,3,4283,2,10.46,2,20.92
99454,I824010,C103292,5,2,4105,3,10.46,2,20.92
99455,I702964,C800631,7,3,4093,5,4200.00,4,16800.00


In [26]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99457 entries, 0 to 99456
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id_factura           99457 non-null  object 
 1   id_cliente           99457 non-null  object 
 2   id_categoria         99457 non-null  int64  
 3   id_metodo_pago       99457 non-null  int64  
 4   id_fecha             99457 non-null  int64  
 5   id_centro_comercial  99457 non-null  int64  
 6   precio_unitario      99457 non-null  float64
 7   cantidad             99457 non-null  int64  
 8   venta_total          99457 non-null  float64
dtypes: float64(2), int64(5), object(2)
memory usage: 6.8+ MB


# Cargar datos

In [27]:
# Cargar datos procesados en la tabla de ventas
with engine.begin() as conn:
    df_ventas.to_sql('ventas', con=conn, index=False, if_exists='append')

In [28]:
df_ventas

,id_factura,id_cliente,id_categoria,id_metodo_pago,id_fecha,id_centro_comercial,precio_unitario,cantidad,venta_total
0,I138884,C241288,1,1,4511,1,1500.40,5,7502.00
1,I317333,C111565,2,2,4364,2,1800.51,3,5401.53
2,I127801,C266599,1,3,4272,3,300.08,1,300.08
3,I173702,C988172,2,1,4154,4,3000.85,5,15004.25
4,I337046,C189076,3,3,4315,1,60.60,4,242.40
...,...,...,...,...,...,...,...,...,...
99452,I219422,C441542,8,1,4647,1,58.65,5,293.25
99453,I325143,C569580,5,3,4283,2,10.46,2,20.92
99454,I824010,C103292,5,2,4105,3,10.46,2,20.92
99455,I702964,C800631,7,3,4093,5,4200.00,4,16800.00


# Crear dimensión de clientes y tiempo (fechas)

In [ ]:
# Crear un dataframe con los datos a cargar
dim_clientes = data[['id_cliente', 'genero', 'edad']]
dim_clientes = dim_clientes.rename(columns={'id_cliente': 'id'})
dim_clientes

In [ ]:
# Cargar datos en la tabla de clientes
with engine.begin() as conn:
    dim_clientes.to_sql('clientes', con=conn, index=False, if_exists='append')

In [ ]:
# Crear un rango de fechas desde el 1 de enero de 2010 hasta el 31 de diciembre de 2030
fechas = pd.date_range(start='2010-01-01', end='2030-12-31', freq='D')

# Crear un dataframe con una columna llamada 'fecha_completa' y asignarle las fechas
dim_tiempo = pd.DataFrame({'fecha_completa': fechas})

# Crear columna de id
dim_tiempo['id'] = range(1, len(dim_tiempo) + 1)

# Agregar una columna con el nombre del mes correspondiente a cada fecha en español
dim_tiempo['nombre_mes'] = dim_tiempo['fecha_completa'].dt.month_name(locale = 'es_ES.UTF-8')

# Agregar una columna con el trimestre del año al que pertenece cada fecha
dim_tiempo['trimestre'] = dim_tiempo['fecha_completa'].dt.quarter

# Obtener propiedades de fecha
dim_tiempo['anio'] = dim_tiempo['fecha_completa'].apply(lambda x: x.year)
dim_tiempo['mes'] = dim_tiempo['fecha_completa'].apply(lambda x: x.month)
dim_tiempo['dia'] = dim_tiempo['fecha_completa'].apply(lambda x: x.day)
dim_tiempo['dia_semana'] = dim_tiempo['fecha_completa'].apply(lambda x: x.weekday() + 1)

# Organizar columnas
dim_tiempo = dim_tiempo[['id', 'fecha_completa', 'anio', 'mes', 'dia', 'trimestre', 'nombre_mes', 'dia_semana']]

dim_tiempo

In [ ]:
# Cargar datos en la tabla de fecha
with engine.begin() as conn:
    dim_tiempo.to_sql('fecha', con=conn, index=False, if_exists='append')